In [0]:
#IRS Term Paper Code
#Kenal Butani- 16BIT130
#Rutva Patel- 16BIT129
import re
import seaborn as sns
import string
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline
import numpy
import pandas
import nltk
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import sklearn
# from sklearn import cross_validation
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [0]:
#Load csv file
df = pandas.read_csv("textData.csv",nrows = 1000)
df.head(10)

,Unnamed: 0,sentiment,author,SentimentText,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,1956967341,0,xoshayzers,@tiffanylue i know i was listenin to bad habi...,NaN,NaN,NaN
1,1956967666,0,wannamama,Layin n bed with a headache ughhhh...waitin o...,NaN,NaN,NaN
2,1956967696,0,cool1ky,1eral ceremony...gloomy friday...,NaN,NaN,NaN
3,1956967789,1,czareaquino,wants to hang out with friends SOON!,NaN,NaN,NaN
4,1956968416,1,xkilljoyx,@dannycastillo We want to trade with someone w...,NaN,NaN,NaN
5,1956968477,0,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,NaN,NaN,NaN
6,1956968487,0,ShansBee,"I should be sleep, but im not! thinking about ...",NaN,NaN,NaN
7,1956968636,0,mcsleazy,Hmmm. http://www.djhero.com/ is down,NaN,NaN,NaN
8,1956969035,0,nic0lepaula,@charviray Charlene my 1. I miss you,NaN,NaN,NaN
9,1956969172,0,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,NaN,NaN,NaN


In [0]:
#1.Removing unicode
result = []
for text in df['SentimentText']:
    text = re.sub(r'(\\u [0-9A-Za-z]+)',r'', text)       
    text = re.sub(r'[^\x00-\x7f]',r'',text)
    result.append(text)
df['SentimentText'] = result
df['SentimentText'] = df['SentimentText'].str.replace("[^a-zA-Z0-9#!?.@']", " ")
df['SentimentText'].head(5)

0    @tiffanylue i know  i was listenin to bad habi...
1    Layin n bed with a headache  ughhhh...waitin o...
2                    1eral ceremony...gloomy friday...
3                 wants to hang out with friends SOON!
4    @dannycastillo We want to trade with someone w...
Name: SentimentText, dtype: object

In [0]:
#2.Replacing user names and urls by AT_USER and URL
result = []
for text in df['SentimentText']:
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',text)
    text = re.sub('@[^\s]+','AT_USER',text)
    text = re.sub('#[^\s]+','',text)
    result.append(text)
df['SentimentText'] = result
df['SentimentText'].head(5)

0    AT_USER i know  i was listenin to bad habit ea...
1    Layin n bed with a headache  ughhhh...waitin o...
2                    1eral ceremony...gloomy friday...
3                 wants to hang out with friends SOON!
4    AT_USER We want to trade with someone who has ...
Name: SentimentText, dtype: object

In [0]:
#3.removing abbrevations
d = {' AFAIK ':' As Far As I Know ',' AFK ':' Away From Keyboard ',' ASAP ':' As Soon As Possible ',' ATK ':' At The Keyboard ',' ATM ':' At The Moment ',' A3 ':' Anytime, Anywhere, Anyplace ',' BAK ':' Back At Keyboard ',' BBL ':' Be Back Later ',' BBS ':' Be Back Soon ',' BFN ':' Bye For Now ',' B4N ':' Bye For Now ',' BRB ':' Be Right Back ',' BRT ':' Be Right There ',' BTW ':' By The Way ',' B4 ':' Before ','B4N':'Bye For Now','CU':'See You','CUL8R':'See You Later','CYA':'See You','FAQ':'Frequently Asked Questions','FC':'Fingers Crossed','FWIW':'For What It is Worth','FYI':'For Your Information','GAL':'Get A Life','GG':'Good Game','GN':'Good Night','GMTA':'Great Minds Think Alike','GR8':'Great!','G9':'Genius','IC':'I See','ICQ':'I Seek you','ILU':'I Love You','IMHO':'In My Honest/Humble Opinion','IMO':'In My Opinion','IOW':'In Other Words','IRL':'In Real Life','KISS':'Keep It Simple, Stupid','LDR':'Long Distance Relationship','LMAO':'Laugh My A.. Off','LOL':'Laughing Out Loud','LTNS':'Long Time No See','L8R':'Later','MTE':'My Thoughts Exactly','M8':'Mate','NRN':'No Reply Necessary','OMG':'Oh My God','OIC':'Oh I See','PITA':'Pain In The A..','PRT':'Party','PRW':'Parents Are Watching','QPSA?':'Que Pasa?','ROFL':'Rolling On The Floor Laughing','ROFLOL':'Rolling On The Floor Laughing Out Loud','ROTFLMAO':'Rolling On The Floor Laughing My A.. Off','SK8':'Skate','STATS':'Your sex and age','ASL':'Age, Sex, Location','THX':'Thank You','TTFN':'Ta-Ta For Now!','TTYL':'Talk To You Later',' U ':' You ','U2':'You Too','U4E':'Yours For Ever','WB':'Welcome Back','WTF':'What The F...','WTG':'Way To Go!','WUF':'Where Are You From?','W8':'Wait...','7K':'Sick'}
result = []
for text in df['SentimentText']:
    for word in text.split():
        if word in d:
            text = text.replace(word, d[word])
    result.append(text)
df['SentimentText'] = result
df['SentimentText'].head(5)

0    AT_USER i know  i was listenin to bad habit ea...
1    Layin n bed with a headache  ughhhh...waitin o...
2                    1eral ceremony...gloomy friday...
3                 wants to hang out with friends SOON!
4    AT_USER We want to trade with someone who has ...
Name: SentimentText, dtype: object

In [0]:
#4.Replacing contractions
contractions = {"ain't": "am not / are not","aren't": "are not / am not","can't": "cannot","can't've": "cannot have","'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he had / he would","he'd've": "he would have","he'll": "he shall / he will","he'll've": "he shall have / he will have","he's": "he has / he is","how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how has / how is","i'd": "I had / I would","i'd've": "I would have","i'll": "I shall / I will","i'll've": "I shall have / I will have","i'm": "I am","i've": "I have","isn't": "is not","it'd": "it had / it would","it'd've": "it would have","it'll": "it shall / it will","it'll've": "it shall have / it will have","it's": "it has / it is","let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have","she'd": "she had / she would","she'd've": "she would have","she'll": "she shall / she will","she'll've": "she shall have / she will have","she's": "she has / she is","should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so as / so is","that'd": "that would / that had","that'd've": "that would have","that's": "that has / that is","there'd": "there had / there would","there'd've": "there would have","there's": "there has / there is","they'd": "they had / they would","they'd've": "they would have","they'll": "they shall / they will","they'll've": "they shall have / they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had / we would","we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have","weren't": "were not","what'll": "what shall / what will","what'll've": "what shall have / what will have","what're": "what are","what's": "what has / what is","what've": "what have","when's": "when has / when is","when've": "when have","where'd": "where did","where's": "where has / where is","where've": "where have","who'll": "who shall / who will","who'll've": "who shall have / who will have","who's": "who has / who is","who've": "who have","why's": "why has / why is","why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all","y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had / you would","you'd've": "you would have","you'll": "you shall / you will","you'll've": "you shall have / you will have","you're": "you are","you've": "you have"}
result = []
for text in df['SentimentText']:
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    result.append(text)
df['SentimentText'] = result
df['SentimentText'].head(5)

0    AT_USER i know  i was listenin to bad habit ea...
1    Layin n bed with a headache  ughhhh...waitin o...
2                    1eral ceremony...gloomy friday...
3                 wants to hang out with friends SOON!
4    AT_USER We want to trade with someone who has ...
Name: SentimentText, dtype: object

In [0]:
#5.Removing Numbers
result = []
for text in df['SentimentText']:
    text = ''.join([i for i in text if not i.isdigit()])
    result.append(text)
df['SentimentText'] = result
df['SentimentText'].head(5)


0    AT_USER i know  i was listenin to bad habit ea...
1    Layin n bed with a headache  ughhhh...waitin o...
2                     eral ceremony...gloomy friday...
3                 wants to hang out with friends SOON!
4    AT_USER We want to trade with someone who has ...
Name: SentimentText, dtype: object

In [0]:
#6. Replacing multiple punctuations
result = []
for text in df['SentimentText']:
    text = re.sub(r"(\!)\1+", ' multiExclamation ', text)
    text = re.sub(r"(\?)\1+", ' multiQuestion ', text)
    text = re.sub(r"(\.)\1+", ' multiStop ', text)
    result.append(text)
df['SentimentText'] = result
df['SentimentText'].head(5)


0    AT_USER i know  i was listenin to bad habit ea...
1    Layin n bed with a headache  ughhhh multiStop ...
2     eral ceremony multiStop gloomy friday multiStop 
3                 wants to hang out with friends SOON!
4    AT_USER We want to trade with someone who has ...
Name: SentimentText, dtype: object

In [0]:
#7. Replacing negations with antonyms
from nltk.corpus import wordnet
def replace(word, pos=None):
    antonyms = set()
    for syn in wordnet.synsets(word, pos=pos):
      for lemma in syn.lemmas():
        for antonym in lemma.antonyms():
          antonyms.add(antonym.name())
    if len(antonyms) == 1:
      return antonyms.pop()
    else:
      return None

def replaceNegations(text):
    i, l = 0, len(text)
    words = []
    while i < l:
      word = text[i]
      if word == 'not' and i+1 < l:
        ant = replace(text[i+1])
        if ant:
          words.append(ant)
          i += 2
          continue
      words.append(word)
      i += 1
    return words

result = []
for text in df['SentimentText']:
    text = replaceNegations(text)
    result.append(text)
for i in range(len(df['SentimentText'])):
    df['SentimentText'][i] = ''.join(df['SentimentText'][i])
df['SentimentText'] = result
for i in range(len(df['SentimentText'])):
    df['SentimentText'][i] = ''.join(df['SentimentText'][i])
df['SentimentText'].head(5)

/home/kenal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kenal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    AT_USER i know  i was listenin to bad habit ea...
1    Layin n bed with a headache  ughhhh multiStop ...
2     eral ceremony multiStop gloomy friday multiStop 
3                 wants to hang out with friends SOON!
4    AT_USER We want to trade with someone who has ...
Name: SentimentText, dtype: object

In [0]:
#8. Removing punctuation
result = []
for text in df['SentimentText']:
    text = re.sub(r'[^\w\s]','',text)
    result.append(text)
df['SentimentText'] = result
df['SentimentText'].head(5)


0    AT_USER i know  i was listenin to bad habit ea...
1    Layin n bed with a headache  ughhhh multiStop ...
2     eral ceremony multiStop gloomy friday multiStop 
3                  wants to hang out with friends SOON
4    AT_USER We want to trade with someone who has ...
Name: SentimentText, dtype: object

In [0]:
#10. Lowercasing
result = []
for text in df['SentimentText']:
    text = text.lower()
    result.append(text)
df['SentimentText'] = result
df['SentimentText'].head(5)


0    at_user i know  i was listenin to bad habit ea...
1    layin n bed with a headache  ughhhh multistop ...
2     eral ceremony multistop gloomy friday multistop 
3                  wants to hang out with friends soon
4    at_user we want to trade with someone who has ...
Name: SentimentText, dtype: object

In [0]:
#11. Removing stop words
df['SentimentText'] = df['SentimentText'].apply(lambda x: x.split())
from nltk.corpus import stopwords
result = []
for text in df['SentimentText']:
    text = [w for w in text if not w in stopwords.words("english")]
    result.append(text)
for i in range(len(df['SentimentText'])):
    df['SentimentText'][i] = ''.join(df['SentimentText'][i])
df['SentimentText'] = result
for i in range(len(df['SentimentText'])):
    df['SentimentText'][i] = ' '.join(df['SentimentText'][i])
df['SentimentText'].head(5)




/home/kenal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/kenal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


0    at_user know listenin bad habit earlier starte...
1    layin n bed headache ughhhh multistop waitin c...
2      eral ceremony multistop gloomy friday multistop
3                              wants hang friends soon
4       at_user want trade someone houston tickets one
Name: SentimentText, dtype: object

In [0]:
#tokenization
df['SentimentText'] = df['SentimentText'].apply(lambda x: x.split())
df['SentimentText'].head()

0    [at_user, know, listenin, bad, habit, earlier,...
1    [layin, n, bed, headache, ughhhh, multistop, w...
2    [eral, ceremony, multistop, gloomy, friday, mu...
3                         [wants, hang, friends, soon]
4    [at_user, want, trade, someone, houston, ticke...
Name: SentimentText, dtype: object

In [0]:
#12. Stemming
from nltk.stem.porter import *
stemmer = PorterStemmer()
df['SentimentText'] = df['SentimentText'].apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
for i in range(len(df['SentimentText'])):
    df['SentimentText'][i] = ' '.join(df['SentimentText'][i])
df['SentimentText'].head(5)

/home/kenal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    at_us know listenin bad habit earlier start fr...
1    layin n bed headach ughhhh multistop waitin ca...
2      eral ceremoni multistop gloomi friday multistop
3                                want hang friend soon
4           at_us want trade someon houston ticket one
Name: SentimentText, dtype: object

In [0]:
labels = numpy.asarray(df.sentiment)
df_features = df['SentimentText']
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
vec.fit(df_features)
features = vec.transform(df_features)
features=features.toarray()

In [0]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.20, random_state=3, shuffle=True) 
print('train on %d instances, test on %d instances' % (len(features_train), len(features_test)))

train on 80 instances, test on 20 instances


In [0]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=23)
# train the classifier using the training data
clf.fit(features_train, labels_train)
#predict class
labels_pred = clf.predict(features_test)
# compute accuracy using test data
from sklearn.metrics import accuracy_score
acc_test = accuracy_score(labels_test, labels_pred)
print ("Test Accuracy:", acc_test)

Test Accuracy: 0.8


In [0]:
from sklearn.svm import SVC
clf = SVC(kernel="linear")
# train the classifier using the training data
clf.fit(features_train, labels_train)
#predict class
labels_pred = clf.predict(features_test)
# compute accuracy using test data
from sklearn.metrics import accuracy_score
acc_test = accuracy_score(labels_test, labels_pred)
print ("Test Accuracy:", acc_test)

Test Accuracy: 0.75


In [0]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
# train the classifier using the training data
clf.fit(features_train, labels_train)
#predict class
labels_pred = clf.predict(features_test)
# compute accuracy using test data
from sklearn.metrics import accuracy_score
acc_test = accuracy_score(labels_test, labels_pred)
print ("Test Accuracy:", acc_test)

Test Accuracy: 0.75


In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
# train the classifier using the training data
clf.fit(features_train, labels_train)
#predict class
labels_pred = clf.predict(features_test)
# compute accuracy using test data
from sklearn.metrics import accuracy_score
acc_test = accuracy_score(labels_test, labels_pred)
print ("Test Accuracy:", acc_test)

Test Accuracy: 0.65


In [0]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=998)
# train the classifier using the training data
clf.fit(features_train, labels_train)
#predict class
labels_pred = clf.predict(features_test)
# compute accuracy using test data
from sklearn.metrics import accuracy_score
acc_test = accuracy_score(labels_test, labels_pred)
print ("Test Accuracy:", acc_test)

Test Accuracy: 0.75
